In [1]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable
import time
import collections
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

from tqdm import tqdm
import numpy as np
import jax
import jax.numpy as jnp
import jax.lax as lax
from jaxtyping import Array, Float, Int, PRNGKeyArray
import aeon
import pandas as pd
from preprocessing.timeseries_augmentation import normalize_mean_std_traindata, normalize_streams, augment_time, add_basepoint_zero
from aeon.regression.sklearn import RotationForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from aeon.datasets.tser_datasets import tser_soton
from aeon.datasets import load_regression, load_classification
from sklearn.linear_model import RidgeCV

from features.sig import SigTransform, LogSigTransform
from features.base import TimeseriesFeatureTransformer, TabularTimeseriesFeatures, RandomGuesser, RandomProjectionFeatures
from features.sig_neural import RandomizedSignature, TimeInhomogenousRandomizedSignature
from features.SWIM_controlled_resnet import SampledControlledResNet
from features.efficient_SCRN import memory_efficient_SCRN
from features.rocket_wrappers import RocketWrapper
from utils.utils import print_name, print_shape

jax.config.update('jax_platform_name', 'gpu') # Used to set the platform (cpu, gpu, etc.)
np.set_printoptions(precision=3, threshold=5) # Print options

/home/nikita/Code/zephyrox/.conda/lib/python3.10/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(
2024-09-30 16:32:21.416283: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.5.82). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


# Download dataset code

In [2]:
def get_aeon_dataset(
        dataset_name:str,
        normalize:bool = True,
        TSER_or_MTSC:str = "TSER",
        extract_path = "/home/nikita/hdd/Data/",
        max_T:int = 1000,
        ):
    """Loads a dataset from the UCR/UEA archive using
    the aeon library.

    Args:
        dataset_name (str): Name of the dataset
        normalize (bool): Whether to apply z-score normalization based on
            the train set data.
        TSER_or_MTSC (str): Whether to load a TSER or MTSC dataset.
        extract_path (str): Path to the directory where the datasets are stored.
            Note that the datasets are stored in a subdirectory called "TSER" or "MTSC".
        max_T (int): Maximum length of the time series. If the time series is longer
            than this value, it will be average pooled down.

    Returns:
        Tuple: 4-tuple of the form (X_train, y_train, X_test, y_test)
    """
    if TSER_or_MTSC == "TSER":
        load_fun = load_regression 
    elif TSER_or_MTSC == "MTSC":
        load_fun = load_classification
    else:
        raise ValueError("TSER_or_MTSC must be either 'TSER' or 'MTSC'")
    
    #download
    X_train, y_train = load_fun(dataset_name, split="train", extract_path=extract_path+TSER_or_MTSC+"/")
    X_test, y_test = load_fun(dataset_name, split="test", extract_path=extract_path+TSER_or_MTSC+"/")

    #to jax array NOTE hardcoded no gradients
    X_train = lax.stop_gradient(jnp.array(X_train).transpose(0,2,1))
    X_test = lax.stop_gradient(jnp.array(X_test).transpose(0,2,1))
    y_train = lax.stop_gradient(jnp.array(y_train))[:, None]
    y_test = lax.stop_gradient(jnp.array(y_test))[:, None]

    #normalize
    if normalize:
        X_train, X_test = normalize_streams(X_train, X_test, max_T)
        y_train, y_test = normalize_mean_std_traindata(y_train, y_test)
        c = 5.0
        X_train = X_train.clip(-c, c)
        X_test = X_test.clip(-c, c)

    return X_train, y_train, X_test, y_test


# df = []
# for dataset_name in list(tser_soton):
#     print(dataset_name)
#     X_train, y_train, X_test, y_test = get_aeon_dataset(dataset_name)

#     N1, T, D = X_train.shape
#     N2, T, D = X_test.shape
#     metadata_row = {
#                     "Dataset": dataset_name,
#                     "N_train": N1,
#                     "N_test": N2,
#                     "T": T,
#                     "D": D,
#                     "y_train_shape": y_train.shape,
#                     "n_labels": len(np.unique(y_train)),}
#     df.append(metadata_row)
# df = pd.DataFrame(df).set_index("Dataset")
# df.to_pickle("TSER_dataset_metadata.pkl")


In [3]:
df_meta = pd.read_pickle("TSER_dataset_metadata.pkl")
df_meta.head()

,N_train,N_test,T,D,y_train_shape,n_labels
Dataset,,,,,,
FloodModeling1,471,202,266,1,"(471, 1)",85
BarCrawl6min,140,61,360,3,"(140, 1)",137
TetuanEnergyConsumption,254,110,144,5,"(254, 1)",254
GasSensorArrayEthanol,324,140,938,1,"(324, 1)",4
FloodModeling3,429,184,266,1,"(429, 1)",76


In [4]:
df_meta.sort_values("T").iloc[0:23]

,N_train,N_test,T,D,y_train_shape,n_labels
Dataset,,,,,,
ParkingBirmingham,1391,597,14,1,"(1391, 1)",1385
NaturalGasPricesSentiment,65,28,20,1,"(65, 1)",65
CardanoSentiment,74,33,24,2,"(74, 1)",66
BeijingPM25Quality,11918,5048,24,9,"(11918, 1)",575
EthereumSentiment,249,107,24,2,"(249, 1)",241
MetroInterstateTrafficVolume,849,365,24,4,"(849, 1)",838
BeijingPM10Quality,11918,5048,24,9,"(11918, 1)",710
BitcoinSentiment,232,100,24,2,"(232, 1)",228
AustraliaRainfall,112186,48081,24,3,"(112186, 1)",782


In [5]:
short_datasets = df_meta.query("T <= 100").sort_values("N_train", ascending=False)
print(short_datasets)
short_datasets = list(short_datasets.index)

                               N_train  N_test    T  D y_train_shape  n_labels
Dataset                                                                       
AustraliaRainfall               112186   48081   24  3   (112186, 1)       782
VentilatorPressure               52815   22635   80  2    (52815, 1)     38384
ElectricMotorTemperature         15503    6645   60  6    (15503, 1)     15502
BeijingPM10Quality               11918    5048   24  9    (11918, 1)       710
BeijingPM25Quality               11918    5048   24  9    (11918, 1)       575
MethaneMonitoringHomeActivity     2017     865  100  1     (2017, 1)      2017
LPGasMonitoringHomeActivity       2017     865  100  1     (2017, 1)      2017
DhakaHourlyAirQuality             1447     621   24  1     (1447, 1)      1255
ParkingBirmingham                 1391     597   14  1     (1391, 1)      1385
WaveDataTension                   1325     568   57  1     (1325, 1)      1325
BeijingIntAirportPM25Quality      1099     472   24 

In [6]:
large_datasets = df_meta.query("N_train >= 3000").sort_values("N_train")
print(large_datasets)
large_datasets = list(large_datasets.index)

                          N_train  N_test     T  D y_train_shape  n_labels
Dataset                                                                   
BenzeneConcentration         3349    5163   240  8     (3349, 1)      2194
LiveFuelMoistureContent      3493    1510   365  7     (3493, 1)       796
MadridPM10Quality            4845    2077   168  3     (4845, 1)      4825
BIDMC32RR                    5471    2399  1000  2     (5471, 1)       502
BIDMC32SpO2                  5550    2399  1000  2     (5550, 1)       281
BIDMC32HR                    5550    2399  1000  2     (5550, 1)      1373
BeijingPM10Quality          11918    5048    24  9    (11918, 1)       710
BeijingPM25Quality          11918    5048    24  9    (11918, 1)       575
ElectricMotorTemperature    15503    6645    60  6    (15503, 1)     15502
DailyTemperatureLatitude    27440   11760   365  1    (27440, 1)       954
PPGDalia                    43215   21482   256  4    (43215, 1)     43211
VentilatorPressure       

In [7]:
medium_datasets = df_meta.query("1000 < N_train < 3000").sort_values("N_train")
print(medium_datasets)
medium_datasets = list(medium_datasets.index)

                               N_train  N_test     T  D y_train_shape  \
Dataset                                                                 
BeijingIntAirportPM25Quality      1099     472    24  6     (1099, 1)   
WaveDataTension                   1325     568    57  1     (1325, 1)   
ParkingBirmingham                 1391     597    14  1     (1391, 1)   
DhakaHourlyAirQuality             1447     621    24  1     (1447, 1)   
MagnesiumConcentration            1560     669   895  1     (1560, 1)   
PotassiumConcentration            1561     669   895  1     (1561, 1)   
PhosphorusConcentration           1573     675   895  1     (1573, 1)   
IEEEPPG                           1768    1328  1000  5     (1768, 1)   
MethaneMonitoringHomeActivity     2017     865   100  1     (2017, 1)   
LPGasMonitoringHomeActivity       2017     865   100  1     (2017, 1)   

                               n_labels  
Dataset                                  
BeijingIntAirportPM25Quality        970

In [8]:
small_datasets = df_meta.query("N_train <= 1000").sort_values("N_train")
print(small_datasets)
small_datasets = list(small_datasets.index)

                              N_train  N_test    T   D y_train_shape  n_labels
Dataset                                                                       
NaturalGasPricesSentiment          65      28   20   1       (65, 1)        65
CardanoSentiment                   74      33   24   2       (74, 1)        66
AppliancesEnergy                   95      42  144  24       (95, 1)        93
DailyOilGasPrices                 133      58   30   2      (133, 1)       133
Covid3Month                       140      61   84   1      (140, 1)       110
BarCrawl6min                      140      61  360   3      (140, 1)       137
Covid19Andalusia                  142      62   91   1      (142, 1)       142
AcousticContaminationMadrid       166      72  365   1      (166, 1)       114
BinanceCoinSentiment              184      79   24   2      (184, 1)       173
SteamPredictor                    210      90  168   4      (210, 1)       210
BitcoinSentiment                  232     100   24  

# Experiment code

In [9]:
from features.efficient_SCRN import memory_efficient_SCRN


def lambda_TSModel(X_train, y_train, X_test, TSModel, **model_args,):
    t0 = time.time()
    model = TSModel(**model_args)
    model.fit(X_train, y_train)
    t1 = time.time()
    feat_train = model.transform(X_train)
    feat_test = model.transform(X_test)
    t2 = time.time()
    return feat_train, feat_test, t1-t0, t2-t1



def run_ridge(
        feat_train: Float[Array, "N1  F"],
        feat_test: Float[Array, "N2  F"],
        y_train: Float[Array, "N1  1"],
        y_test: Float[Array, "N2  1"],
        normalize_features: bool = False,
        ):
    if normalize_features:
        feat_train, feat_test = normalize_mean_std_traindata(feat_train, feat_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    feat_train = np.array(feat_train)
    feat_test = np.array(feat_test)

    #fit ridge regression
    t0 = time.time()
    ridge = RidgeCV(alphas=np.logspace(-3, 3, 30))
    ridge.fit(feat_train, y_train)

    # predict
    pred_test = ridge.predict(feat_test)
    pred_train = ridge.predict(feat_train)
    test_rmse = np.sqrt(mean_squared_error(y_test, pred_test))
    train_rmse = np.sqrt(mean_squared_error(y_train, pred_train))
    alpha = ridge.alpha_
    t1 = time.time()
    return train_rmse, test_rmse, alpha, t1-t0



def run_allmodels_1dataset(
        X_train: Float[Array, "N1  T  D"],
        y_train: Float[Array, "N1  1"],
        X_test: Float[Array, "N2  T  D"],
        y_test: Float[Array, "N2  1"],
        normalize_features: bool = False,
        ):
    prng_key = jax.random.PRNGKey(999)
    max_batch = 10000
    n_features = 100

   # (name, model, model_args)
    TS_models = [
        ["No-Skill", RandomGuesser, {"seed": prng_key}],
        ["Tabular", TabularTimeseriesFeatures, {}],
        ["Random Projection", RandomProjectionFeatures, {"prng_key": prng_key, "n_features": n_features}],
        ["Sig", SigTransform, {"trunc_level": 3}],
        ["Randomized Signature", RandomizedSignature, 
            {"seed": prng_key, "n_features": n_features, "max_batch": max_batch}],
        ["Time-Inhomogenous Randomized Signature", TimeInhomogenousRandomizedSignature, 
            {"seed": prng_key, "n_features": n_features, "max_batch": max_batch}],
        ["Rocket", RocketWrapper, {"n_features": n_features}],
    ]
    custom_models = [
        ["SCRN", memory_efficient_SCRN, {"seed":prng_key, "n_features":n_features, "activation":jnp.tanh}],
        ["SCRN no dx", memory_efficient_SCRN, {"seed":prng_key, "n_features":n_features, "activation":jnp.tanh, "without_dx":True}],
    ]
    results = []
    model_names = [name for (name, _, _) in TS_models+custom_models]

    # TSFeature models
    for name, model, model_args in TS_models:
        print(name)
        feat_train, feat_test, t_fit, t_feat = lambda_TSModel(X_train, y_train, X_test, model, **model_args)
        res = run_ridge(feat_train, feat_test, y_train, y_test, normalize_features)
        results.append( res + (t_fit, t_feat) )

    # Other models
    for name, model_fun, model_args in custom_models:
        print(name)
        feat_train, feat_test, t_fit, t_feat = model_fun(X_train, y_train, X_test, **model_args)
        res = run_ridge(feat_train, feat_test, y_train, y_test, normalize_features)
        results.append( res + (t_fit, t_feat) )

    return model_names, results



def run_all_experiments(
        datasets: List[str],
        name_save: str,
        normalize_X: bool = True,
        normalize_features: bool = False,
        normalize_y: bool = True,
        apply_augment_time: bool = False,
        apply_basepoint: bool = False,
        max_T = 100,
        ):
    # Run experiments
    experiments = {}
    for dataset_name in tqdm(datasets):
        print(dataset_name)
        t0 = time.time()

        # augment dataset
        X_train, y_train, X_test, y_test = get_aeon_dataset(dataset_name, normalize_X, max_T=max_T)
        if apply_basepoint:
            X_train = add_basepoint_zero(X_train)
            X_test = add_basepoint_zero(X_test)
        if apply_augment_time:
            X_train = augment_time(X_train)
            X_test  = augment_time(X_test)
        if normalize_y:
            y_train, y_test = normalize_mean_std_traindata(y_train, y_test)

        results = run_allmodels_1dataset(
            X_train, y_train, X_test, y_test, 
            normalize_features,
            )
        experiments[dataset_name] = results
        print(dataset_name, "Elapsed time", time.time()-t0)
    
    # Save results
    # Assuming experiments is a dict where keys are dataset names and values are tuples (model_names, results)
    attributes = ["RMSE_train", "RMSE_test", "alpha", "t_ridge", "t_fit", "t_feat"]
    data_list = []
    # Process the data
    for dataset_name, (model_names, results) in experiments.items():
        dataset_data = {}
        for attr_idx, attribute in enumerate(attributes):
            for model_idx, model_name in enumerate(model_names):
                dataset_data[(attribute, model_name)] = results[model_idx][attr_idx]
        data_list.append(pd.DataFrame(dataset_data, index=[dataset_name]))

    # Combine all datasets into a single DataFrame
    df = pd.concat(data_list)
    df = df.sort_index(axis=1)
    print(df)
    df.to_pickle(f"TSER_results_{name_save}.pkl")
    return df

In [10]:
df = run_all_experiments(short_datasets, "short", max_T=1000)

  0%|          | 0/22 [00:00<?, ?it/s]

AustraliaRainfall
No-Skill
Tabular
Random Projection
Sig
Randomized Signature
Time-Inhomogenous Randomized Signature
Rocket
SCRN
SCRN no dx


  5%|▍         | 1/22 [01:31<32:11, 91.97s/it]

AustraliaRainfall Elapsed time 91.96653366088867
VentilatorPressure
No-Skill
Tabular
Random Projection
Sig
Randomized Signature
Time-Inhomogenous Randomized Signature
Rocket
SCRN
SCRN no dx


  9%|▉         | 2/22 [02:28<23:43, 71.19s/it]

...

DailyOilGasPrices Elapsed time 4.830331325531006
CardanoSentiment
No-Skill
Tabular
Random Projection
Sig
Randomized Signature
Time-Inhomogenous Randomized Signature
Rocket
SCRN
SCRN no dx


 95%|█████████▌| 21/22 [05:02<00:04,  4.63s/it]

CardanoSentiment Elapsed time 3.842646837234497
NaturalGasPricesSentiment
No-Skill
Tabular
Random Projection
Sig
Randomized Signature
Time-Inhomogenous Randomized Signature
Rocket
SCRN
SCRN no dx


100%|██████████| 22/22 [05:06<00:00, 13.94s/it]

NaturalGasPricesSentiment Elapsed time 4.335830211639404
                              RMSE_test                    \
                               No-Skill Random Projection   
AustraliaRainfall              0.412829          0.397116   
VentilatorPressure             1.001244          0.653850   
ElectricMotorTemperature       1.023459          0.627272   
BeijingPM10Quality             1.116990          0.811588   
BeijingPM25Quality             1.106576          0.676051   
MethaneMonitoringHomeActivity  1.037485          0.990246   
LPGasMonitoringHomeActivity    1.053946          1.016315   
DhakaHourlyAirQuality          1.061065          0.198009   
ParkingBirmingham              1.113577          1.000558   
WaveDataTension                1.134550          0.822036   
BeijingIntAirportPM25Quality   1.020134          0.754810   
MetroInterstateTrafficVolume   0.999762          0.949391   
SierraNevadaMountainsSnow      0.936217          0.324669   
EthereumSentiment           

In [22]:
df["t_feat"].mean()

No-Skill                                  0.221095
Random Projection                         0.081428
Randomized Signature                      0.719368
Rocket                                    0.947372
SCRN                                      1.260110
SCRN no dx                                0.966185
Sig                                       0.558333
Tabular                                   0.148067
Time-Inhomogenous Randomized Signature    0.447082
dtype: float64

In [23]:
df["RMSE_test"].mean()

No-Skill                                  1.027432
Random Projection                         0.784925
Randomized Signature                      0.815121
Rocket                                    0.803780
SCRN                                      0.823229
SCRN no dx                                0.730923
Sig                                       0.825078
Tabular                                   0.785025
Time-Inhomogenous Randomized Signature    0.835613
dtype: float64

In [24]:
df["RMSE_test"].rank(axis=1).mean()

No-Skill                                  8.954545
Random Projection                         4.318182
Randomized Signature                      5.272727
Rocket                                    4.727273
SCRN                                      5.272727
SCRN no dx                                2.090909
Sig                                       4.318182
Tabular                                   4.136364
Time-Inhomogenous Randomized Signature    5.909091
dtype: float64

In [25]:
df["RMSE_test"].rank(axis=1)

,No-Skill,Random Projection,Randomized Signature,Rocket,SCRN,SCRN no dx,Sig,Tabular,Time-Inhomogenous Randomized Signature
AustraliaRainfall,9.0,7.0,8.0,5.0,3.0,1.0,2.0,6.0,4.0
VentilatorPressure,9.0,4.0,6.0,2.0,7.0,1.0,8.0,3.0,5.0
ElectricMotorTemperature,9.0,6.0,4.0,8.0,3.0,2.0,1.0,5.0,7.0
BeijingPM10Quality,9.0,4.0,6.0,7.0,8.0,3.0,1.0,2.0,5.0
BeijingPM25Quality,9.0,3.0,6.0,8.0,7.0,1.0,4.0,2.0,5.0
MethaneMonitoringHomeActivity,9.0,7.0,8.0,4.0,1.0,2.0,3.0,6.0,5.0
LPGasMonitoringHomeActivity,9.0,7.0,6.0,3.0,2.0,1.0,5.0,8.0,4.0
DhakaHourlyAirQuality,9.0,4.0,6.0,8.0,2.0,1.0,7.0,3.0,5.0
ParkingBirmingham,9.0,7.0,8.0,3.0,2.0,1.0,5.0,6.0,4.0
WaveDataTension,9.0,2.0,7.0,6.0,4.0,5.0,8.0,1.0,3.0


In [15]:
df["RMSE_test"]

,No-Skill,Random Projection,Randomized Signature,Rocket,SCRN,SCRN no dx,Sig,Tabular,Time-Inhomogenous Randomized Signature
AustraliaRainfall,0.412829,0.397116,0.408729,0.395622,0.395111,0.391607,0.393954,0.397038,0.395377
VentilatorPressure,1.001244,0.653850,0.778387,0.583626,0.825944,0.547822,0.965612,0.653839,0.747537
ElectricMotorTemperature,1.023459,0.627272,0.625455,0.635449,0.598410,0.559614,0.454676,0.626835,0.631114
BeijingPM10Quality,1.116990,0.811588,0.856059,0.912791,0.913153,0.806079,0.799571,0.800749,0.854803
BeijingPM25Quality,1.106576,0.676051,0.745838,0.815682,0.754438,0.659524,0.711442,0.669849,0.713546
MethaneMonitoringHomeActivity,1.037485,0.990246,0.995525,0.977621,0.919218,0.920899,0.948073,0.990137,0.984868
LPGasMonitoringHomeActivity,1.053946,1.016315,1.009372,1.007811,0.979300,0.975913,1.009222,1.017887,1.009182
DhakaHourlyAirQuality,1.061065,0.198009,0.321189,0.449238,0.189249,0.071920,0.439072,0.198005,0.266961
ParkingBirmingham,1.113577,1.000558,1.020680,0.804992,0.772768,0.750293,0.974300,1.000529,0.879234
WaveDataTension,1.134550,0.822036,0.919409,0.916983,0.877296,0.879709,0.988892,0.821681,0.872344


In [16]:
df_med = pd.read_pickle("TSER_results_medium4.pkl")
df_med["RMSE_test"].mean()

No-Skill                                  1.083999
Random Projection                         0.777899
Randomized Signature                      0.924689
Rocket                                    0.734427
Sampled CRN                               0.765049
Sampled CRN no dx                         0.761567
Sig                                       1.081864
Tabular                                   0.781244
Time-Inhomogenous Randomized Signature    0.852990
dtype: float64

In [17]:
df_med["RMSE_test"].rank(axis=1).mean()

No-Skill                                  8.9
Random Projection                         3.9
Randomized Signature                      6.6
Rocket                                    3.8
Sampled CRN                               3.2
Sampled CRN no dx                         2.6
Sig                                       6.3
Tabular                                   4.5
Time-Inhomogenous Randomized Signature    5.2
dtype: float64

In [18]:
df_med["RMSE_test"].rank(axis=1)

,No-Skill,Random Projection,Randomized Signature,Rocket,Sampled CRN,Sampled CRN no dx,Sig,Tabular,Time-Inhomogenous Randomized Signature
BeijingIntAirportPM25Quality,9.0,4.0,8.0,3.0,2.0,1.0,6.0,5.0,7.0
WaveDataTension,9.0,1.0,7.0,6.0,5.0,4.0,8.0,2.0,3.0
ParkingBirmingham,9.0,6.0,8.0,1.0,3.0,2.0,4.0,7.0,5.0
DhakaHourlyAirQuality,9.0,5.0,7.0,6.0,1.0,2.0,8.0,4.0,3.0
MagnesiumConcentration,9.0,1.0,8.0,3.0,5.0,4.0,7.0,2.0,6.0
PotassiumConcentration,9.0,2.0,8.0,1.0,5.0,4.0,7.0,3.0,6.0
PhosphorusConcentration,9.0,3.0,7.0,1.0,5.0,4.0,8.0,2.0,6.0
IEEEPPG,8.0,5.0,4.0,1.0,2.0,3.0,9.0,6.0,7.0
MethaneMonitoringHomeActivity,9.0,6.0,4.0,8.0,2.0,1.0,3.0,7.0,5.0
LPGasMonitoringHomeActivity,9.0,6.0,5.0,8.0,2.0,1.0,3.0,7.0,4.0


In [19]:
df_med["RMSE_test"]

,No-Skill,Random Projection,Randomized Signature,Rocket,Sampled CRN,Sampled CRN no dx,Sig,Tabular,Time-Inhomogenous Randomized Signature
BeijingIntAirportPM25Quality,1.043678,0.741420,0.868496,0.738821,0.724861,0.713736,0.840296,0.748680,0.845856
WaveDataTension,1.128268,0.811683,0.985453,0.907404,0.879852,0.879709,0.988892,0.821681,0.839360
ParkingBirmingham,1.134294,1.000519,1.040897,0.693742,0.764364,0.761416,0.974300,1.000532,0.980735
DhakaHourlyAirQuality,1.066792,0.198276,0.415107,0.361582,0.068519,0.072169,0.439072,0.198016,0.195287
MagnesiumConcentration,1.028923,0.256666,0.896982,0.317083,0.519140,0.519130,0.852167,0.262943,0.644585
PotassiumConcentration,0.813227,0.556810,0.701939,0.477486,0.636582,0.636532,0.698511,0.557108,0.658213
PhosphorusConcentration,0.979867,0.738516,0.909390,0.567931,0.845507,0.845494,0.910774,0.730585,0.884634
IEEEPPG,1.555274,1.468991,1.434139,1.248261,1.290486,1.290674,3.157338,1.484871,1.486745
MethaneMonitoringHomeActivity,1.036580,0.989480,0.985227,0.999248,0.922297,0.920899,0.948073,0.990137,0.985227
LPGasMonitoringHomeActivity,1.053091,1.016627,1.009258,1.032708,0.998877,0.975913,1.009222,1.017887,1.009257
